In [4]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf
import happybase
import warnings
import sklearn.model_selection
import sklearn.preprocessing
import pickle
import csv
import json

In [5]:
# data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"
data_dir = r"/home/zhaodachuan/data/predict_user_profile20200911"

In [6]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

['/home/zhaodachuan/data/predict_user_profile20200911/000000_0', '/home/zhaodachuan/data/predict_user_profile20200911/000002_0', '/home/zhaodachuan/data/predict_user_profile20200911/000003_0', '/home/zhaodachuan/data/predict_user_profile20200911/000001_0']


In [7]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list
old_df = copy.deepcopy(df)

ValueError: Length mismatch: Expected axis has 3 elements, new values have 32 elements

In [5]:
df = copy.deepcopy(old_df)

In [6]:
# 获得指定列
use_column_list = ["用户id","用户性别","作者类型",'关注的作者','关注的马甲']
y_col = "用户性别"
y_col_transfer_dict = {"1":"男","2":"女"}
id_col = "用户id"
df = df[use_column_list]

In [7]:
df.fillna("",inplace=True)
df

,用户id,用户性别,作者类型,关注的作者,关注的马甲
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1,-1
...,...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1,-1


# 数据分析

In [ ]:
df.info()

In [ ]:
#df["微信性别"].unique()

In [ ]:
df["作者类型"].unique()

In [ ]:
df["关注的马甲"].unique()

In [ ]:
df["关注的作者"].unique()

# 训练模型

In [8]:
#处理性别列
df[y_col] = df[y_col].apply(lambda x : str(x))
df[y_col].replace(y_col_transfer_dict,inplace=True)
df

,用户id,用户性别,作者类型,关注的作者,关注的马甲
0,00001D26FE991A4BC4A01325D5E222DE,-1,-1,-1,-1
1,00006C3EF7F79B0F37EEFB8381505530,-1,-1,-1,-1
2,0000D0E1960239E8362F6B4100D9790A,-1,-1,-1,-1
3,0000f55e60d5ba7b36004429b235c95096f7153d,-1,-1,-1,-1
4,00014E9D5CF948B3FE6376564B23F64F,-1,-1,-1,-1
...,...,...,...,...,...
15703260,qlu2xsthqk7tp0mncpgr,-1,-1,-1,-1
15703261,qlu30bysbp1mj150uc1m,-1,-1,-1,-1
15703262,qlu30t7n6f2b1jgzexv,-1,-1,-1,-1
15703263,qlu315b37v5rc273amb0,-1,-1,-1,-1


In [9]:
def deal_with_sex(sex,y_set=set(y_col_transfer_dict.values())):
    if sex in y_set:
        return True
    return False

predict_df = df[-df[y_col].apply(deal_with_sex)]
train_df = df[df[y_col].apply(deal_with_sex)]
train_df

,用户id,用户性别,作者类型,关注的作者,关注的马甲
10293,100092,男,-1,-1,-1
10541,10018,男,-1,-1,-1
10739,10025,男,-1,-1,-1
11009,100346,男,-1,-1,-1
12469,10087,男,-1,-1,-1
...,...,...,...,...,...
14643469,99300,男,-1,-1,-1
14643527,99319,男,-1,-1,-1
14643950,99450,男,-1,-1,-1
14644686,99685,男,-1,-1,-1


In [15]:
import happybase
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(train_df.index):
            batch.put(str(train_df.loc[index,"用户id"]).encode(), {"attribute:input_gender".encode():train_df.loc[index,y_col].encode()})
            #print(str(train_df.loc[index,"用户id"]).encode(),{"attribute:input_gender".encode():train_df.loc[index,y_col].encode()})
            #break

In [ ]:
predict_df

In [ ]:
#训练数据
multi_hot_column_odict = collections.OrderedDict()

multi_hot_column_set = set([v for v in use_column_list if v != y_col and v!=id_col])
for i,column in enumerate(multi_hot_column_set):
    multi_hot_column_odict[column] = sklearn.preprocessing.MultiLabelBinarizer()
    split_column_list = [item.split(",") if isinstance(item,str) else str(item) for item in train_df[column]]
    #multi_hot_column_odict[column].fit(split_column_list)
    train_df[column] = multi_hot_column_odict[column].fit_transform(split_column_list).tolist()

In [ ]:
%%time
y_onehot = sklearn.preprocessing.LabelBinarizer()
y_onehot.fit(train_df[y_col])
train_df[y_col] = y_one_hot.transform(train_df[y_col])

In [ ]:
train_df

In [ ]:
train, test = sklearn.model_selection.train_test_split(train_df, test_size=0.2)
train, val = sklearn.model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
def generate_xy(data_df, y_col, multi_hot_column_odict, shuffle=True):
    """"""
    x_df = data_df.copy()
    x_list = []
    y_list = []
    for i,index in enumerate(x_df.index):
        temp_x_list = np.concatenate(x_df.loc[index,[column for column in multi_hot_column_odict]]).tolist()
        temp_y_list = x_df.loc[index,y_col]
        x_list.append(temp_x_list)
        y_list.append(temp_y_list)
        if i % 10000 == 0:
            print(i)
    return np.array(x_list,dtype=np.float32), np.array(y_list,dtype=np.int32)

In [ ]:
%%time
x_train,y_train = generate_xy(train,y_col,multi_hot_column_odict)
x_vali,y_valid = generate_xy(val,y_col,multi_hot_column_odict)
x_test,y_test = generate_xy(test,y_col,multi_hot_column_odict)

In [ ]:
for k,v in multi_hot_column_odict.items():
    print(k,len(v.classes_))

In [ ]:
x_dim = 0
for k,v in  multi_hot_column_odict.items():
    x_dim += len(v.classes_)
print(x_dim)
print(y_onehot.classes_)
y_dim = 1 if len(y_onehot.classes_) == 2 else len(y_onehot.classes_)
print(y_dim)

In [ ]:
class TfKerasLrModel(x_dim,y_dim):
    """keras创建模型
    """
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(input_dim,)))
    if y_dim == 2:
        model.add(
            tf.keras.layers.Dense(
                len(y_onehot.categories_), 
                activation='softmax',use_bias=True,
                #kernel_regularizer=tf.keras.regularizers.l1(0.01)
            )
        )
    elif y_dim > 2:
        model.add(
            tf.keras.layers.Dense(
                len(y_onehot.categories_), 
                activation='softmax',use_bias=True,
                #kernel_regularizer=tf.keras.regularizers.l1(0.01)
            )
        )
    else:
        raise Exception("分类类别错误，需要 >=2 ，获得{}".format(y_dim))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

In [ ]:
model = TfKerasLrModel()
model.summary()

In [ ]:
x_train

In [ ]:
collections.Counter([str(v) for v in y_train])

In [ ]:
model.fit(x_train, 
          y_train, 
          batch_size=64,
          epochs=20,
         validation_data = (x_vali,y_valid))

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
#保存模型
with open("multi_hot_column_odict.pkl","wb") as f:
    pickle.dump(y_onehot,f)
with open("y_onehot.pkl","wb") as f:
    pickle.dump(y_onehot,f)
model.save("model_output")

In [ ]:
predict_df

In [ ]:
multi_hot_column_odict

In [ ]:
%%time
#测试数据
warnings.filterwarnings('ignore')
#output_csv = r"/mnt1/zhangwanli/user_profile_output/gender.csv"
output_csv = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/gender.csv"
with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['gender', 'udid' , "probility"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i,chunk in enumerate(np.array_split(predict_df, 1000)):
        udid_list = predict_df["用户id"].values.tolist()
        predict_x = []
        for index in chunk.index:
            print(index)
            break
        if i == 3:
            break

In [ ]:
%%time
#测试数据
warnings.filterwarnings('ignore')
#output_csv = r"/mnt1/zhangwanli/user_profile_output/gender.csv"
output_csv = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/gender.csv"
with open(output_csv, 'w', newline='') as csvfile:
    fieldnames = ['gender', 'udid' , "probility"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i,chunk in enumerate(np.array_split(predict_df, 1000)):
        udid_list = chunk["用户id"].values.tolist()
        predict_x = []
        for index in chunk.index:
            temp_x_list = []
            for j,column in enumerate(multi_hot_column_odict):
                split_column_list = chunk.loc[index,column].split(",") if isinstance(chunk.loc[index,column],str) else [str(predict_df.loc[index,column])]
                temp_x_list.extend(multi_hot_column_odict[column].transform([split_column_list]).tolist()[0])
            predict_x.append(temp_x_list)
        y_predict = model.predict([predict_x])
        for y,y_predict_i,udid in zip(np.argmax(y_predict[0],axis=1),y_predict[0],udid_list):
            writer.writerow({'gender':y, 'udid': udid , 'probility':json.dumps(y_predict_i.tolist())})
        print(i)

In [ ]:
y_transform_dict = {
    "0":"男",
    "1":"女",
}
output_df

In [ ]:
output_df = pd.read_csv("/mnt1/zhaodachuan/data/predict_user_attribute/data/gender.csv")
output_df

In [ ]:
collections.Counter(output_df['gender'])

In [ ]:
import happybase
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(output_df.index):
            batch.put(str(output_df.loc[index,"udid"]).encode(), {"attribute:predict_gender".encode(): y_transform_dict[str(output_df.loc[index,"gender"])].encode()})
            #break
            if i % 100000 == 0:
                print(i)